
# 03 – Valorisation d'options : formule fermée, Monte Carlo et EDP

Ce notebook illustre la **chaîne de pricing** :
- formule de Black–Scholes (fermée) ;
- Monte Carlo générique pour des payoffs plus complexes ;
- solveur EDP par différences finies ;
- comparaison des résultats et des Greeks.


In [1]:

import pathlib, sys
import numpy as np

NOTEBOOK_DIR = pathlib.Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))

from pricing.black_scholes import bs_call, bs_put
from pricing.monte_carlo import mc_price_european   # adapter au nom réel
from pricing.pde_solver import crank_nicolson_bs  # adapter au nom réel
from pricing.greeks import greeks_fd     # adapter aux fonctions réelles



## 1. Paramètres d'une option européenne

On considère un call européen simple pour comparer les méthodes.


In [2]:

S0 = 100.0
K = 100.0
T = 1.0
r = 0.01
sigma = 0.2

# Paramètres Monte Carlo
N = 252      # pas de temps
M = 50_000   # nombre de trajectoires



## 2. Prix Black–Scholes (formule fermée)


In [3]:

bs_price = bs_call(S0=S0, K=K, T=T, r=r, sigma=sigma)
print("Prix call Black–Scholes (fermée) :", bs_price)


Prix call Black–Scholes (fermée) : 8.433318690109608



## 3. Prix Monte Carlo

On estime
$$ V_0 \approx e^{-rT} \frac{1}{M} \sum_{m=1}^M \Phi(S_{\cdot}^{(m)}). $$


In [4]:

def payoff_call(paths, K):
    # payoff en fonction de la trajectoire : ici, call européen standard
    ST = paths[:, -1]
    return np.maximum(ST - K, 0.0)

from models.gbm import GBM

model = GBM(mu=r, sigma=sigma, S0=S0)


mc_price, mc_std = mc_price_european(
    model=model,
    K=K,
    T=T,
    r=r,
    M=M,
    payoff="call",
    antithetic=True,
    N=N,
    random_state=42,
)

print("Prix call Monte Carlo      :", mc_price)
print("Erreur standard Monte Carlo:", mc_std)


Prix call Monte Carlo      : 8.43119118749257
Erreur standard Monte Carlo: 0.11859796734199295



## 4. Prix par EDP (différences finies)

On résout numériquement l'équation de Black–Scholes en temps inverse.


In [5]:

Smax = 4 * S0  # borne supérieure de la grille

S_grid, V_grid = crank_nicolson_bs(
    Smax=Smax,
    K=K,
    T=T,
    r=r,
    sigma=sigma,
    M_S=400,
    M_T=1000,
    option="call",
)

# Prix au spot S0 par interpolation linéaire
pde_price = np.interp(S0, S_grid, V_grid)
print("Prix call via EDP (PDE) :", pde_price)


Prix call via EDP (PDE) : 8.43087106696193



## 5. Comparaison des résultats

On regarde les écarts relatifs entre les différentes méthodes.


In [6]:

def rel_diff(a, b):
    return abs(a - b) / max(1e-8, abs(a))

print("Écart MC vs BS   :", rel_diff(bs_price, mc_price))
print("Écart PDE vs BS  :", rel_diff(bs_price, pde_price))


Écart MC vs BS   : 0.00025227347562880497
Écart PDE vs BS  : 0.00029023249774135017



## 6. Greeks (sensibilités)

On calcule Delta, Gamma (et éventuellement Vega, Theta, Rho) pour analyser la couverture.


In [7]:

greeks = greeks_fd(S0, K, T, r, sigma, payoff="call")
delta_bs = greeks["delta"]
gamma_bs = greeks["gamma"]
vega_bs  = greeks["vega"]
rho_bs   = greeks["rho"]
theta_bs = greeks["theta"]

print("Delta (BS) :", delta_bs)
print("Gamma (BS) :", gamma_bs)


Delta (BS) : 0.5596176866173863
Gamma (BS) : 0.019723966246942837



### Idées d'analyse

- Tracer l'erreur Monte Carlo en fonction de $M$ (convergence en $1/\sqrt{M}$).
- Étudier la sensibilité du prix à $\sigma$ (Vega) et la comparer aux formules analytiques.
- Comparer les profils de Delta obtenus via différences finies sur la grille de prix.
